In [ ]:
import importlib
import brikasutils as bu
importlib.reload(bu)
import shared_utils as utils
from shared_utils import systemMsg, userMsg, assistantMsg
importlib.reload(utils)
import survey
importlib.reload(survey)
import persona
importlib.reload(persona)
import ollama
import numpy as np
from numpy.linalg import norm
import pandas as pd
import os
import re
import time
import json
from typing import List
from openai import OpenAI

In [ ]:
et = persona.PersonaEncoder()
et.parse_fb_messages(["data/1_raw/1_airidas.json"], "airidas")
et.parse_fb_messages(["data/1_raw/2_christian.json"], "christian")
et.parse_fb_messages(["data/1_raw/1_nikolay.json"], "nikolay")
et.parse_fb_messages(["data/1_raw/2_mathis.json"], "mathis")
et.parse_fb_messages(["data/1_raw/2_jacob.json"], "jacob")
et.parse_fb_messages(["data/1_raw/2_chris.json"], "chris")
et.parse_fb_messages(["data/1_raw/3_aziz.json"], "aziz")
et.parse_fb_messages(["data/1_raw/3_daniela.json"], "daniela")
et.parse_fb_messages(["data/1_raw/3_mihi.json"], "mihi")
et.parse_fb_messages(["data/1_raw/3_viktoria.json"], "viktoria")
et.parse_fb_messages(["data/1_raw/4_diba.json"], "diba")
et.parse_fb_messages(["data/1_raw/6_filip.json"], "filip")
et.parse_wa_messages(["data/1_raw/messages_1000.json"], "rebecca")
et.filter_chats_empty()
et.filter_chats_regex(utils.BLACKLIST_CHAT_REGEX_FILTERS)
for nameid, chat in et.chats.items():
    for msg in chat:  
        msg.sender = "Persona" if msg.sender == "Elias Salvador Smidt Torjani"  else "Friend"
et.select_chat_full("rebecca")
et.select_chat_full("airidas")
et.select_chat_full("christian")
et.select_chat_full("nikolay")
et.select_chat_full("mathis") 
et.select_chat_full("daniela")
et.select_chat_full("diba")
et.select_chat_full("aziz")
et.select_chat_full("jacob")  
et.select_chat_full("chris")
et.select_chat_full("filip")
et.select_chat_full("mihi")
et.select_chat_full("viktoria")

### Proof of concept run

In [ ]:
# fixed variables (7b model, static, impersonate, 3 runs each)
EMBED_MODEL = "nomic-embed-text"
MODEL = "llama3:7B"
PROMPT_METHOD ="IMPERSONATE"
SUBJECT = "Elias"
RETRIAVAL_METHOD = "static"
# SURVEY_PROMPT = "Determine how much {subject} aggree with the statement. Guestimate how {subject} would answer to the question"
# TINY_MODULE = "You are Elias, a 24 year old business and IT student from Copenhagen, where you now live in a dormatory."
# "MED_MODULE": " "
# "PERSONA_TEXT": "Favorite video games are Minecraft, Fortnite, and Call of Duty.",

surv = survey.PersonalitySurvey()
if isinstance(surv, survey.KanoSurvey):
    DYNAMIC_RETRIEVAL_PROMPTS = list(surv.questions)
    PROMPT_COUNT = 40
    SURVEY_TYPE = "KanoSurvey",
    WHICH_SURVEY = "kano"
    RETRIEVAL_PROMPT = "video game features"
    SURVEY = "video game preferences"
    METHOD = "Kano survey"
elif isinstance(surv, survey.PersonalitySurvey):
    DYNAMIC_RETRIEVAL_PROMPTS = list(surv.questions)
    PROMPT_COUNT = 50
    SURVEY_TYPE = "PersonalitySurvey",
    WHICH_SURVEY = "pers"
    RETRIEVAL_PROMPT = "openess conciousness extrovert aggreableness neuroticism"
    SURVEY = "personality traits"
    METHOD = "OCEAN test"

In [ ]:
CHUNK_SIZES = [10, 20, 30]
OVERLAP_SIZES = [1, 3, 5]
CHUNKS_COUNT_IN_CTX = [1, 2, 3]
# RETRIAVAL_METHODS = ["static", "dynamic"]
# SURVEYS = [survey.KanoSurvey(), survey.PersonalitySurvey()]
# CHUNK_ORDER_IN_CTX = ["CHRONOLOGICAL", "REVERSE"]

# 27 iterations below
for chunk_size in CHUNK_SIZES:
    chunks = []
    for chat in et.selectedChats.values():
        messages = list(chat)  # Convert chat iterator to list for easier slicing
        num_messages = len(messages)
        for i in range(0, num_messages - chunk_size + 1, chunk_size - overlap_size):
            chunk = messages[i:i + chunk_size]  # Extract chunk of messages
            chunk_text = "\n".join(str(msg) for msg in chunk)  # Concatenate msgs into a single string
            chunks.append(chunk_text)  # Append chunk to list of chunks
            
    for overlap_size in OVERLAP_SIZES:
        embeddings = []
        progress, chunks_len = 0, len(chunks) # for progress bar
        for chunk_text in chunks:
            progress += 1
            print(f"\rChunk {progress}/{chunks_len}", end="")
            embedding = ollama.embeddings(model=EMBED_MODEL, prompt=chunk_text)["embedding"]
            embeddings.append(embedding)
            
        for chunks_count_in_ctx in CHUNKS_COUNT_IN_CTX:
            SIM_ID = f"{chunk_size}-{overlap_size}-{chunks_count_in_ctx}-{RETRIAVAL_METHOD}"
            
            final_prompts = []
            if RETRIAVAL_METHOD == "static":
                prompt_embedding = ollama.embeddings(model=EMBED_MODEL, prompt=RETRIEVAL_PROMPT)["embedding"]
                chunks_most_similar_embeddings  = utils.find_most_similar(prompt_embedding, embeddings)[:chunks_count_in_ctx]
                chunks_most_similar = []
                for embedding in chunks_most_similar_embeddings:
                    chunks_most_similar.append(chunks[embedding[1]])
                tokens_in_chunks = 0
                for chunk in chunks_most_similar:
                    tokens_in_chunks += utils.count_tokens(chunk)
                print(f"Tokens in chunks: {tokens_in_chunks}")
                #Retrieve^
                prompt_template = """
for question in surv.questions:
    p = [
        systemMsg("\\n".join([
            f"You are an expert actor, specializing in impersonation of non-famouns people. You will be presented to the subject through explicit datapoints of their digital footprint. In addition, you will deduct their implicit {SURVEY} by shadowing chats between the subject and friends. You will be asked to fully immerse yourself in the role, and answer questions from the point of view of the persona. \\n\\n**The persona, which you will be tasked to mimick is named '{SUBJECT}'.** \\n#Context \\n##Chat conversations between the subject and their friends:\\n**From most to least related**\\n",
            "\\nNEW CONVERSATION:\\n".join(chunks_most_similar)
        ])),  
        assistantMsg("Understood. I will answer from the point of view of the persona, {subject}, based on what I could the deduct from the text provided."),
        userMsg("\\n".join([
            f"Persona is questioned about their {SURVEY} in an {METHOD}. The persona must choose an appropriate answer to the question below with one of these five given options: {', '.join(surv.POSSIBLE_ANSWERS)}. Persona's answer must only contain the chosen option, without any elaboration, nor introduction.\\n\\n**Your question is:**\\n\\n",
            question,
            "\\nThe persona chooses:"
        ]))]
    final_prompts.append(p)
                """

#             elif RETRIAVAL_METHOD == "dynamic":
#                 dynamic_retrieval_prompts = list(surv.questions)
#                 dynamic_chunks_most_similar: List[List[str]] = [] 

#                 progress = 0
#                 lenn = len(dynamic_retrieval_prompts)
#                 for prompt in dynamic_retrieval_prompts:
#                     progress += 1
#                     print(f"\rPrompt {progress}/{lenn}", end="")

#                     prompt_embedding = ollama.embeddings(model=EMBED_MODEL, prompt=prompt)["embedding"]
#                     chunks_most_similar_embeddings = utils.find_most_similar(prompt_embedding, embeddings)[:CHUNKS_COUNT_IN_CTX]
#                     chunks_most_similar = []
#                     for embedding in chunks_most_similar_embeddings:
#                         chunks_most_similar.append(chunks[embedding[1]])

#                     dynamic_chunks_most_similar.append(chunks_most_similar)
#                 print(end="\n")
#                 tokens_in_chunks = 0
#                 for chunks_most_similar in dynamic_chunks_most_similar:
#                     for chunk in chunks_most_similar:
#                         tokens_in_chunks += utils.count_tokens(chunk)

#                 del chunks_most_similar_embeddings # free memory
#                 print(f"Tokens in average chunk group: {tokens_in_chunks/len(dynamic_chunks_most_similar)}")
#                 #Retrieve^
#                 prompt_template = """
# for question, chunks_most_similar in zip(surv.questions, dynamic_chunks_most_similar):
# p = [
#     systemMsg("\\n".join([
#         f"You are an expert actor, specializing in impersonation of non-famouns people. You will be presented to the subject through explicit datapoints of their digital footprint. In addition, you will deduct their implicit {SURVEY} by shadowing chats between the subject and friends. You will be asked to fully immerse yourself in the role, and answer questions from the point of view of the persona. \\n\\n**The persona, which you will be tasked to mimick is named '{SUBJECT}'.** \\n#Context \\n##Chat conversations between the subject and their friends:\\n**From most to least related**\\n",
#         "\\nNEW CONVERSATION:\\n".join(chunks_most_similar)
#     ])),      
#     assistantMsg("Understood. I will answer from the point of view of the persona, {subject}, based on what I could the deduct from the text provided."),
#     userMsg("\\n".join([
#         f"Persona is questioned about their {SURVEY} in an {METHOD}. The persona must choose an appropriate answer to the question below with one of these five given options: {', '.join(surv.POSSIBLE_ANSWERS)}. Persona's answer must only contain the chosen option, without any elaboration, nor introduction.\\n\\n**Your question is:**\\n\\n",
#         question,
#         "\\nThe persona chooses:"
#     ]))]
# final_prompts.append(p)
#                 """

            else: print("couldn't ID retrieval method")
            
            exec(prompt_template)
            prompt_info = utils.describe_prompts_and_print(final_prompts) # Vanity print
            bu.quickJSON(final_prompts, f"data/5_monster_prep/{SIM_ID}_prompts.json")

            instructions = {
                "prompt_file": f"batch/prompts/{SIM_ID}_prompt.json",
                "survey_type": f"{SURVEY_TYPE}",
                "isLocal": True,
                "LIMIT": None
            }
            settings = {
                "model": MODEL,
                "timeout": 300
            }
            AUTO_INFO = {
                "CHUNK_SIZE": chunk_size,
                "OVERLAP_SIZE": overlap_size,
                "tokens_in_chunks": tokens_in_chunks,
                "CHUNKS_COUNT_IN_CTX": chunks_count_in_ctx,
                "model": EMBED_MODEL,
                "prompt method": PROMPT_METHOD,
                "retrieval method": RETRIAVAL_METHOD,
                "retrieval prompt": RETRIEVAL_PROMPT,
                "prompt_count": PROMPT_COUNT,
                "survey": WHICH_SURVEY,
                "subject": SUBJECT,
                "prompt_template": prompt_template,
                **utils.describe_prompts([])
            }
            bu.quickJSON({"instructions": instructions, "settings": settings, "info": AUTO_INFO}, f"data/5_monster_prep/batch-schema/{SIM_ID}_prompts.json")

### V2 WIP

In [ ]:
CHUNK_SIZES = [10, 20, 30]
OVERLAP_SIZES = [1, 3, 5]
max_tokens = 7000

for chunk_size in CHUNK_SIZES:
    chunks = []
    chunk_token_counts = []  # Initialize a list to store token counts of each chunk

    for chat in et.selectedChats.values():
        messages = list(chat)  # Convert chat iterator to list for easier slicing
        num_messages = len(messages)
        for i in range(0, num_messages - chunk_size + 1, chunk_size - overlap_size):
            chunk = messages[i:i + chunk_size]  # Extract chunk of messages
            chunk_text = "\\n".join(str(msg) for msg in chunk)  # Concatenate msgs into a single string
            chunks.append(chunk_text)  # Append chunk to list of chunks
            chunk_token_counts.append(utils.count_tokens(chunk_text))  # Append token count of the chunk

    avg_chunk_token_count = sum(chunk_token_counts) / len(chunk_token_counts)  # Calculate average token count per chunk

    for overlap_size in OVERLAP_SIZES:
        embeddings = []
        progress, chunks_len = 0, len(chunks)
        for chunk_text in chunks:
            progress += 1
            print(f"\n Chunk {progress}/{chunks_len}", end="")
            embedding = ollama.embeddings(model=EMBED_MODEL, prompt=chunk_text)["embedding"]
            embeddings.append(embedding)

        if RETRIAVAL_METHOD == "static":
            prompt_embedding = ollama.embeddings(model=EMBED_MODEL, prompt=RETRIEVAL_PROMPT)["embedding"]
            tokens_in_prompt = utils.count_tokens(RETRIEVAL_PROMPT)
            tokens_remaining = max_tokens - tokens_in_prompt
            # Calculate the maximum number of chunks that can fit within the context window
            max_chunks_count = int(tokens_remaining / avg_chunk_token_count)
            # Retrieve the most similar chunks based on embeddings
            chunks_most_similar_embeddings = utils.find_most_similar(prompt_embedding, embeddings)[:max_chunks_count]
            chunks_most_similar = [chunks[embedding[1]] for embedding in chunks_most_similar_embeddings]
            # Calculate the actual token count of the selected chunks
            tokens_in_chunks = sum(utils.count_tokens(chunk) for chunk in chunks_most_similar)
            # ... (code to process the chunks)

######
# 9 iterations below
# for chunk_size in CHUNK_SIZES:
#     chunks = []
#     for chat in et.selectedChats.values():
#         messages = list(chat)  # Convert chat iterator to list for easier slicing
#         num_messages = len(messages)
#         for i in range(0, num_messages - chunk_size + 1, chunk_size - overlap_size):
#             chunk = messages[i:i + chunk_size]  # Extract chunk of messages
#             chunk_text = "\n".join(str(msg) for msg in chunk)  # Concatenate msgs into a single string
#             chunks.append(chunk_text)  # Append chunk to list of chunks

#     for overlap_size in OVERLAP_SIZES:
#         embeddings = []
#         progress, chunks_len = 0, len(chunks)
#         for chunk_text in chunks:
#             progress += 1
#             print(f"\rChunk {progress}/{chunks_len}", end="")
#             embedding = ollama.embeddings(model=EMBED_MODEL, prompt=chunk_text)["embedding"]
#             embeddings.append(embedding)

#         if RETRIAVAL_METHOD == "static":
#             prompt_embedding = ollama.embeddings(model=EMBED_MODEL, prompt=RETRIEVAL_PROMPT)["embedding"]
#             chunks_and_embeddings = list(zip(chunks, embeddings))

#             # Sort chunks and embeddings by token count
#             chunks_and_embeddings.sort(key=lambda x: utils.count_tokens(x[0]))

#             tokens_remaining = max_tokens - utils.count_tokens(RETRIEVAL_PROMPT)
#             chunks_most_similar = []
#             for chunk, embedding in chunks_and_embeddings:
#                 chunk_tokens = utils.count_tokens(chunk)
#                 if chunk_tokens <= tokens_remaining:
#                     tokens_remaining -= chunk_tokens
#                     chunks_most_similar.append(chunk)
#                 else:
#                     break

#             tokens_in_chunks = max_tokens - tokens_remaining
#             max_chunks_count = len(chunks_most_similar)

#             print(f"Tokens in chunks: {tokens_in_chunks}")
#             print(f"Chunks count in context: {max_chunks_count}")


#             ###########################
#             prompt_embedding = ollama.embeddings(model=EMBED_MODEL, prompt=RETRIEVAL_PROMPT)["embedding"]
#             chunks_most_similar_embeddings  = utils.find_most_similar(prompt_embedding, embeddings)[:chunks_count_in_ctx]
#             chunks_most_similar = []
#             for embedding in chunks_most_similar_embeddings:
#                 chunks_most_similar.append(chunks[embedding[1]])
#             tokens_in_chunks = 0
#             for chunk in chunks_most_similar:
#                 tokens_in_chunks += utils.count_tokens(chunk)
#             #Retrieve^===============================================
#             chunks_most_similar_embeddings = utils.find_most_similar(prompt_embedding, embeddings)
#             chunks_most_similar = []
#             tokens_in_chunks = sum(utils.count_tokens(chunk) for chunk in chunks_most_similar)
#             chunk_token_counts = [utils.count_tokens(chunk) for chunk in chunks]
#             avg_chunk_token_count = sum(chunk_token_counts) / len(chunk_token_counts)
#             max_chunks_count = int(max_tokens / avg_chunk_token_count)
#             chunks_most_similar = [chunks[embedding[1]] for embedding in chunks_most_similar_embeddings[:max_chunks_count]]
            
            # print(f"Tokens in chunks: {tokens_in_chunks} – Chunks count in context: {chunks_count_in_ctx}")
            print(f"AVG Tokens in chunks: {avg_chunk_token_count} – Chunks count in context: {chunks_count_in_ctx}")
            
            #Retrieve^
            prompt_template = """
for question in surv.questions:
    p = [
        systemMsg("\\n".join([
            f"You are an expert actor, specializing in impersonation of non-famouns people. You will be presented to the subject through explicit datapoints of their digital footprint. In addition, you will deduct their implicit {SURVEY} by shadowing chats between the subject and friends. You will be asked to fully immerse yourself in the role, and answer questions from the point of view of the persona. \\n\\n**The persona, which you will be tasked to mimick is named '{SUBJECT}'.** \\n#Context \\n##Chat conversations between the subject and their friends:\\n**From most to least related**\\n",
            "\\nNEW CONVERSATION:\\n".join(chunks_most_similar)
        ])),  
        assistantMsg("Understood. I will answer from the point of view of the persona, {subject}, based on what I could the deduct from the text provided."),
        userMsg("\\n".join([
            f"Persona is questioned about their {SURVEY} in an {METHOD}. The persona must choose an appropriate answer to the question below with one of these five given options: {', '.join(surv.POSSIBLE_ANSWERS)}. Persona's answer must only contain the chosen option, without any elaboration, nor introduction.\\n\\n**Your question is:**\\n\\n",
            question,
            "\\nThe persona chooses:"
        ]))]
    final_prompts.append(p)
        """
        else: print("couldn't ID retrieval method")            
        exec(prompt_template)
        SIM_ID = f"{chunk_size}-{overlap_size}-{chunks_count_in_ctx}-{RETRIAVAL_METHOD}"
        prompt_info = utils.describe_prompts_and_print(final_prompts) # Vanity print
        bu.quickJSON(final_prompts, f"data/5_monster_prep/dynamic-ctx_{SIM_ID}_prompts.json")

        instructions = {
            "prompt_file": f"batch/prompts/{SIM_ID}_prompt.json",
            "survey_type": f"{SURVEY_TYPE}",
            "isLocal": True,
            "LIMIT": None
        }
        settings = {
            "model": MODEL,
            "timeout": 300}
        AUTO_INFO = {
            "CHUNK_SIZE": chunk_size,
            "OVERLAP_SIZE": overlap_size,
            "tokens_in_chunks": tokens_in_chunks,
            "CHUNKS_COUNT_IN_CTX": chunks_count_in_ctx,
            "CTX_limit": max_tokens,
            "model": EMBED_MODEL,
            "prompt method": PROMPT_METHOD,
            "retrieval method": RETRIAVAL_METHOD,
            "retrieval prompt": RETRIEVAL_PROMPT,
            "prompt_count": PROMPT_COUNT,
            "survey": WHICH_SURVEY,
            "subject": SUBJECT,
            "prompt_template": prompt_template,
            "prompt_info": prompt_info,
            **utils.describe_prompts([])
        }
        bu.quickJSON({"instructions": instructions, "settings": settings, "info": AUTO_INFO}, f"data/5_monster_prep/batch-schema/dynamic-ctx_{SIM_ID}_prompts.json")

In [ ]:
##### Display Info
total_messages = sum(len(chat) for chat in et.selectedChats.values())
chunks_count = len(chunks)
avg_chunk_char_len = np.mean([len(chunk) for chunk in chunks])
print(
    f"Chunk count: {chunks_count}",
    f"Rough estimate of tokens per chunk: {round(avg_chunk_char_len / 4)} (4 characters per token)",
    f"Messagees in input count: {total_messages}",
    f"Messages in chunks count: {stat_total_msgs_in_chunks}",
    f"Chunk \ Input ratio: {round(stat_total_msgs_in_chunks / total_messages,2)} (OVERLAP_SIZE={OVERLAP_SIZE})",
    f"Chunk Python type: {type(chunks[0])}",
    sep="\n"
) 
bu.if_dir_not_exist_make("data/3_embeddings")
bu.quickJSON(AUTO_INFO, f"data/3_embeddings/{EMBEDDING_ID}_info.json")
bu.quickJSON({"chunks": chunks, "embeddings": embeddings}, f"data/3_embeddings/{EMBEDDING_ID}_embeddings.json")

In [ ]:
print(f"Tokens in chunks: {tokens_in_chunks}")
print(f"Chunks:{len(chunks)}, embeds:{len(embeddings)}")

# bu.quickTXT("\n\n".join(chunks_most_similar), filename=f"data/4_chunks/{CHECKPOINT}-static_chunks.txt")
# bu.if_dir_not_exist_make("data/4_chunks")
# bu.quickJSON(AUTO_INFO, f"data/4_chunks/{CHECKPOINT}-static_info.json")
# bu.quickJSON({"chunks": chunks, "embeddings": embeddings}, f"data/4_chunks/{CHECKPOINT}-static_embeddings.json")

In [ ]:
# Display results
# bu.if_dir_not_exist_make("data/4_chunks")
# bu.quickJSON(AUTO_INFO, f"data/4_chunks/{CHECKPOINT}-dynamic_info.json")
# bu.quickJSON({"chunks": chunks, "embeddings": embeddings}, f"data/4_chunks/{CHECKPOINT}-dynamic_embeddings.json")
############################################ VANITY BELOW ########################################
tokens_in_chunks = 0
for chunks_most_similar in dynamic_chunks_most_similar:
    for chunk in chunks_most_similar:
        tokens_in_chunks += utils.count_tokens(chunk)

del chunks_most_similar_embeddings # free memory
print(f"Tokens in average chunk group: {tokens_in_chunks/len(dynamic_chunks_most_similar)}")
# bu.quickJSON(dynamic_chunks_most_similar, filename=f"data/4_chunks/{CHECKPOINT}-dynamic_chunks.json")

In [ ]:
########################################### Method A ############################################
pre_final_prompts = []
if PROMPT_METHOD == "IMPERSONATE":
    pre_prompt_template = """
    SYS_MSG = {
        "role": "system", 
        "content": f"You are an expert actor, specializing in impersonation of non-famouns people. You will be presented to the subject through explicit datapoints of their digital footprint. In addition, you will deduct their implicit {SURVEY} by shadowing chats between the subject and friends. You will be asked to fully immerse yourself in the role, and answer questions from the point of view of the persona. \\n\\n**The persona, which you will be tasked to mimick is named '{SUBJECT}'.** \\n#Context \\n##Chat conversations between the subject and their friends:\\n**From most to least related**\\n"
    }
    ASSIST_MSG = {
        "role": "assistant",
        "content": "Understood. I will answer from the point of view of the persona, {subject}, based on what I could the deduct from the text provided."
    }
    USER_MSG = {
        "role": "user",
        "content": f"Persona is questioned about their {SURVEY} in an {METHOD}. The persona must choose an appropriate answer to the question below with one of these five given options: {', '.join(surv.POSSIBLE_ANSWERS)}. Persona's answer must only contain the chosen option, without any elaboration, nor introduction."
    }
    pre_final_prompts.append(SYS_MSG, ASSIST_MSG, USER_MSG)
    """
    exec(pre_prompt_template)

########################################### Method B ###########################################
elif PROMPT_METHOD == "ARE":
    pre_prompt_template = """
    SYS_MSG = {
        "role": "system", 
        "content": f"**{TINY_MODULE}**. You have shared your thoughts, feelings, and experiences through text messages with friedns. Answer the following questions honestly and naturally, as you would in everyday conversations. \\n\\n#Context \\n##Conversations between persona and friends:"
    }
    ASSIST_MSG = {
        "role": "assistant",
        "content": f"Understood. I am {SUBJECT}, and I will answer the survey to the best of my ability."
    }
    USER_MSG = {   
        "role": "user",
        "content": f"The survey is about your {SURVEY}. You must choose an appropriate answer to the question below with one of these five given options: {', '.join(surv.POSSIBLE_ANSWERS)}. Your answer must only contain the chosen option, without any elaboration, nor introduction.\\n**From most to least related**\\n"
    }
    """
exec(pre_prompt_template)

print(f"{SYS_MSG['content']}")

In [ ]:
#BASE
base_final_prompts = []
prompt_template = """
for question in surv.questions:
    p = [
        systemMsg(
            "You are participating in a survey. You will be presented with a series of questions about your {SURVEY}.",
            f"You must choose answer to the question below with one of the five options: {', '.join(surv.POSSIBLE_ANSWERS)}. The answer must only contain the chosen option. "
        ),
        assistantMsg('Understood. I will answer the question below with one of the given options.'),
        userMsg(
            question,
            "Your choice: "
        )]
    base_final_prompts.append(p)
"""
exec(prompt_template)
# prompt_info = utils.describe_prompts_and_print(base_final_prompts) # Vanity print
# bu.quickJSON(base_final_prompts, f"data/5_prep/{WHICH_SURVEY}_base_prompts.json")

utils.count_tokens(prompt_template)
#Static
# static_final_prompts = []
# prompt_template = """
# for question in surv.questions:
#     p = [
#         systemMsg("\\n".join([
#             f"You are an expert actor, specializing in impersonation of non-famouns people. You will be presented to the subject through explicit datapoints of their digital footprint. In addition, you will deduct their implicit {SURVEY} by shadowing chats between the subject and friends. You will be asked to fully immerse yourself in the role, and answer questions from the point of view of the persona. \\n\\n**The persona, which you will be tasked to mimick is named '{SUBJECT}'.** \\n#Context \\n##Chat conversations between the subject and their friends:\\n**From most to least related**\\n",
#             "\\nNEW CONVERSATION:\\n".join(chunks_most_similar)
#         ])),  
#         assistantMsg("Understood. I will answer from the point of view of the persona, {subject}, based on what I could the deduct from the text provided."),
#         userMsg("\\n".join([
#             f"Persona is questioned about their {SURVEY} in an {METHOD}. The persona must choose an appropriate answer to the question below with one of these five given options: {', '.join(surv.POSSIBLE_ANSWERS)}. Persona's answer must only contain the chosen option, without any elaboration, nor introduction.\\n\\n**Your question is:**\\n\\n",
#             question,
#             "\\nThe persona chooses:"
#         ]))]
#     static_final_prompts.append(p)
# """
# exec(prompt_template)
# prompt_info = utils.describe_prompts_and_print(static_final_prompts) # Vanity print
# bu.quickJSON(final_prompts, f"data/5_prep/{PREP_CHECKPOINT}_prompts.json")
# # print(f"{len(static_final_prompts)}")#,{final_prompts[:1]}")

# #Dynamic
# dynamic_final_prompts = []
# prompt_template = """
# for question, chunks_most_similar in zip(surv.questions, dynamic_chunks_most_similar):
#     p = [
#         systemMsg("\\n".join([
#             f"{SYS_MSG['content']}",
#             "\\nNEW CONVERSATION:\\n".join(chunks_most_similar)
#         ])),      
#         assistantMsg(ASSIST_MSG['content']),
#         userMsg("\\n".join([
#             f"{USER_MSG['content']}\\n\\n**Your question is:**\\n\\n",
#             question,
#             "\\nThe persona chooses:"
#         ]))]
#     dynamic_final_prompts.append(p)
# """    
# exec(prompt_template)
# prompt_info = utils.describe_prompts_and_print(dynamic_final_prompts)
# bu.quickJSON(dynamic_final_prompts, f"data/5_prep/{PREP_CHECKPOINT}_prompts.json")
# # print(f"{len(dynamic_final_prompts)}")#,{final_prompts[:1]}")